In [15]:
import wrds
import pandas as pd

# Connect to WRDS
db = wrds.Connection()

# List of the 20 largest companies by ticker symbol 
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'BRK.B', 'NVDA', 'META', 'TSLA', 'V', 
           'JPM', 'UNH', 'MA', 'PG', 'HD', 'DIS', 'VZ', 'PYPL', 'NFLX', 'PEP', 'KO']

# Define the SQL query template
query_template = """
WITH date_range AS (
    SELECT
        CAST(CAST(EXTRACT(YEAR FROM CURRENT_DATE) - 10 AS VARCHAR) || '-01-01' AS DATE) AS start_date,
        CURRENT_DATE AS end_date
)
SELECT
    a.permno,
    b.ticker,
    a.date,
    a.ret,
    a.prc,
    a.vol,
    a.shrout,
    a.prc / a.cfacpr AS adj_prc,  -- Adjusted Price
    a.ret * a.cfacshr AS adj_ret  -- Adjusted Returns
FROM
    crsp.dsf AS a
JOIN
    crsp.dsenames AS b ON a.permno = b.permno
JOIN
    date_range AS dr ON a.date BETWEEN dr.start_date AND dr.end_date
WHERE
    b.ticker = '{ticker}'
    AND a.date >= b.namedt  -- Ensure the stock is active on the date
    AND a.date <= b.nameendt  -- Ensure the stock is active on the date
ORDER BY
    a.permno, a.date;
"""

# Fetch the data for each ticker and combine into a single DataFrame
all_data = []

for ticker in tickers:
    query = query_template.format(ticker=ticker)
    data = db.raw_sql(query)
    all_data.append(data)

# Concatenate all data into a single DataFrame
top_20_companies_stock_data = pd.concat(all_data, ignore_index=True)

# Display the data
top_20_companies_stock_data.head()


Loading library list...
Done


/tmp/20884432.1.jupyterhub.q/ipykernel_74687/1024528768.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  top_20_companies_stock_data = pd.concat(all_data, ignore_index=True)


,permno,ticker,date,ret,prc,vol,shrout,adj_prc,adj_ret
0,14593,AAPL,2014-01-02,-0.014064,553.13000,8338094.0,892447.0,19.754643,-0.393792
1,14593,AAPL,2014-01-03,-0.021966,540.97998,13992006.0,892447.0,19.320714,-0.615048
2,14593,AAPL,2014-01-06,0.005453,543.92999,14820614.0,892447.0,19.426071,0.152684
3,14593,AAPL,2014-01-07,-0.007156,540.03748,11381939.0,892447.0,19.287053,-0.200368
4,14593,AAPL,2014-01-08,0.006338,543.46002,9199689.0,892447.0,19.409286,0.177464


In [16]:
top_20_companies_stock_data.to_csv("data_df.csv")